In [1]:
import warnings
warnings.filterwarnings('ignore')
import plotly.express as px
import pandas as pd
import pyspark.sql.functions as f
df_country_code=pd.read_csv('fips-10-4-to-iso-country-codes.csv')
from pyspark.sql.functions import col
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [2]:
from pyspark.sql import SQLContext
sqlSparkContext = SQLContext(sc)
spark.conf.set("spark.sql.catalog.mycatalog","com.datastax.spark.connector.datasource.CassandraCatalog")

### Donnée du jour

In [45]:
date = sqlSparkContext.sql("SELECT max(date) as day FROM mycatalog.gdelt_prod.importstatus").toPandas()
print(pd.to_datetime(date.day,format='%Y%m%d%H%M%S'))

0   2022-02-11 09:30:00
Name: day, dtype: datetime64[ns]


22/02/11 09:41:01 WARN ChannelPool: [s0|/192.168.3.108:9042]  Error while opening new channel (ConnectionInitException: [s0|connecting...] Protocol initialization request, step 1 (STARTUP {CQL_VERSION=3.0.0, DRIVER_NAME=DataStax Java driver for Apache Cassandra(R), DRIVER_VERSION=4.12.0, CLIENT_ID=620b9e75-6972-404e-83f6-3a513d4929c3, APPLICATION_NAME=Spark-Cassandra-Connector-app-20220211083620-0006}): failed to send request (com.datastax.oss.driver.shaded.netty.channel.StacklessClosedChannelException))
22/02/11 09:41:55 WARN ChannelPool: [s0|/192.168.3.108:9042]  Error while opening new channel (ConnectionInitException: [s0|connecting...] Protocol initialization request, step 1 (STARTUP {CQL_VERSION=3.0.0, DRIVER_NAME=DataStax Java driver for Apache Cassandra(R), DRIVER_VERSION=4.12.0, CLIENT_ID=620b9e75-6972-404e-83f6-3a513d4929c3, APPLICATION_NAME=Spark-Cassandra-Connector-app-20220211083620-0006}): failed to send request (com.datastax.oss.driver.shaded.netty.channel.StacklessClose

### Requête N°1
### Afficher le nombre d’articles/évènements qu’il y a eu pour chaque triplet (jour, pays de l’évènement, langue de l’article).

In [31]:
df1 = sqlSparkContext.sql("SELECT day as date, action_geocountrycode as pays, mentiondoctranslationalinfo as langue,COUNT(DISTINCT globaleventid) as nb_events,COUNT(globaleventid) as nb_articles FROM mycatalog.gdelt_prod.nb_articles_events GROUP BY day, action_geocountrycode,mentiondoctranslationalinfo").show(10)


+--------+----+------+---------+-----------+
|    date|pays|langue|nb_events|nb_articles|
+--------+----+------+---------+-----------+
|20210709|  TT|    NA|        4|          5|
|20220204|  VQ|    NA|        7|          8|
|20210402|  AU|    NA|      342|        390|
|20210306|  GQ|    NA|       37|         37|
|20200829|  LI|    NA|        2|          2|
|20210909|  SW|    NA|      215|        252|
|20210724|  CW|    NA|        2|          2|
|20210207|  EZ|    NA|       53|         57|
|20201028|  YM|    NA|        2|          2|
|20211012|  TZ|    NA|      132|        132|
+--------+----+------+---------+-----------+
only showing top 10 rows



### Requête N°2
### Pour un pays donné en paramètre, affichez les évènements qui y ont eu place triées par le nombre de mentions (tri décroissant); permettez une agrégation par jour/mois/année

In [32]:
country_1=df_country_code[df_country_code['Name']=='Russia']['FIPS 10-4'].values[0]
print(country_1)

RS


In [33]:
country=country_1
temps="day"

In [7]:
sqlSparkContext.sql("SELECT {}, globaleventid,nummentions, action_geocountrycode FROM mycatalog.gdelt_prod.countries_events WHERE action_geocountrycode='{}' GROUP BY {},globaleventid,nummentions,action_geocountrycode ORDER BY nummentions DESC".format(temps,country,temps)).show(10)


+--------+-------------+-----------+---------------------+
|     day|globaleventid|nummentions|action_geocountrycode|
+--------+-------------+-----------+---------------------+
|20211231|   1021307781|        260|                   RS|
|20211231|   1021307778|        202|                   RS|
|20211231|   1021306706|        202|                   RS|
|20211231|   1021307790|        180|                   RS|
|20211231|   1021306914|        180|                   RS|
|20211231|   1021307448|        180|                   RS|
|20211231|   1021306919|        180|                   RS|
|20211231|   1021306703|        178|                   RS|
|20211228|   1021007361|        170|                   RS|
|20220129|   1025581335|        168|                   RS|
+--------+-------------+-----------+---------------------+
only showing top 10 rows



### Requête N°3
### Pour une source de donnés passée en paramètre (gkg.SourceCommonName) affichez les thèmes, personnes, lieux dont les articles de cette sources parlent ainsi que le nombre d’articles et le ton moyen des articles (pour chaque thème/personne/lieu); permettez une agrégation par jour/mois/année.

In [34]:
source="bbc.com"
agg_time="month"

In [35]:
df3 = sqlSparkContext.sql("SELECT {},sourcecommonname,themes,locations,persons,tone FROM mycatalog.gdelt_prod.data_source WHERE sourcecommonname='{}'".format(agg_time,source))

df_persons = df3.select(agg_time,'sourcecommonname','themes','locations','tone', f.explode("persons").alias("person"))
df_locations = df3.select(agg_time,'sourcecommonname','themes',f.explode("locations").alias("location"),'tone',"persons")
df_themes = df3.select(agg_time,'sourcecommonname',f.explode('themes').alias('theme'),"locations",'tone',"persons")


Themes

In [36]:
df_themes2 = df_themes.groupBy(agg_time,'theme').agg(f.mean('tone'), f.count('theme').alias('Nb_articles'))
df_themes2.sort(col(agg_time).asc() ,col('Nb_articles').desc()).show()

+------+--------------------+-------------------+-----------+
| month|               theme|          avg(tone)|Nb_articles|
+------+--------------------+-------------------+-----------+
|202101|      GENERAL_HEALTH| -2.485387335389139|       1211|
|202101|CRISISLEX_CRISISL...|-3.5989171564970457|       1142|
|202101|WB_621_HEALTH_NUT...|-2.5867267669235146|       1086|
|202101|  GENERAL_GOVERNMENT|-2.8930163343682738|        921|
|202101|     UNGP_HEALTHCARE|  -2.23639224927766|        875|
|202101|              LEADER|  -2.81520936552215|        863|
|202101|             MEDICAL| -2.568336729628016|        848|
|202101|USPEC_POLITICS_GE...|-2.7365415130466864|        789|
|202101|UNGP_FORESTS_RIVE...| -2.148432724943785|        749|
|202101|                KILL|-3.8344763864509086|        748|
|202101|EPU_POLICY_GOVERN...|-2.9364468006282327|        746|
|202101|WB_635_PUBLIC_HEALTH| -2.541889522869715|        745|
|202101|WB_2165_HEALTH_EM...| -2.568507434497307|        664|
|202101|

In [37]:
df_persons2 = df_persons.groupBy(agg_time,'person').agg(f.mean('tone'), f.count('person').alias('Nb_articles'))
df_persons2.sort(col(agg_time).asc() ,col('Nb_articles').desc()).show()

+------+--------------------+-------------------+-----------+
| month|              person|          avg(tone)|Nb_articles|
+------+--------------------+-------------------+-----------+
|202101|no person(s) ment...| -3.004377866533689|        277|
|202101|           joe biden|-2.5040315688096397|        254|
|202101|        donald trump|-2.7241236392343255|        228|
|202101|       boris johnson| -2.428439575025723|        168|
|202101|        barack obama| -1.399427640106943|         45|
|202101|       kamala harris|0.06884972954338248|         44|
|202101|        keir starmer| -2.758029035011003|         43|
|202101|     nicola sturgeon|-2.6692311220935414|         42|
|202101|        matt hancock| -2.502976170182228|         40|
|202101|          mike pence|-3.3520076255767774|         39|
|202101|        nancy pelosi| -4.087905851694254|         39|
|202101|     yoweri museveni| -2.655150863631018|         29|
|202101|      alexei navalny| -7.707979738712311|         28|
|202101|

In [38]:
df_locations2 = df_locations.groupBy(agg_time,'location').agg(f.mean('tone'), f.count('location').alias('Nb_articles'))
df_locations2.sort(col(agg_time).asc() ,col('Nb_articles').desc()).show()

+------+--------------------+-------------------+-----------+
| month|            location|          avg(tone)|Nb_articles|
+------+--------------------+-------------------+-----------+
|202101|      United Kingdom|-1.9477234688805944|        729|
|202101|no location(s) me...|-2.0936765571247857|        265|
|202101|Northern Ireland,...|-1.9367275341483188|        242|
|202101|London, London, C...| -2.199785586956626|        241|
|202101|             British|-1.6999345754121624|        215|
|202101|            American|-1.8023199673127575|        194|
|202101|            Scotland|-2.2107794845008595|        187|
|202101|             America|-2.1627598826397847|        181|
|202101|               China|-2.0731837457993576|        171|
|202101|Washington, Washi...| -2.894411571648897|        149|
|202101|       United States|-2.0492125431804555|        115|
|202101|White House, Dist...|-2.6253882733819722|        111|
|202101|               India|-1.7879758395116638|        108|
|202101|

### Requête N°4
### Etudiez l’évolution des relations entre deux pays (specifies en paramètre) au cours de l’année. Vous pouvez vous baser sur la langue de l’article, le ton moyen des articles, les themes plus souvent citées, les personalités ou tout element qui vous semble pertinent


In [39]:
Country_1 = 'France'
country_1=df_country_code[df_country_code['Name']==Country_1]['FIPS 10-4'].values[0]
print(country_1)

Country_2 = 'China'
country_2=df_country_code[df_country_code['Name']==Country_2]['FIPS 10-4'].values[0]
print(country_2)

FR
CH


In [40]:
df1 = sqlSparkContext.sql("SELECT  DISTINCT sourceurl,day, averagetone FROM mycatalog.gdelt_prod.relationship WHERE actor1_geocountrycode='{}' AND actor2_geocountrycode='{}'".format(country_1,country_2))

df2 = sqlSparkContext.sql("SELECT  DISTINCT sourceurl,day, averagetone FROM mycatalog.gdelt_prod.relationship WHERE actor1_geocountrycode='{}' AND actor2_geocountrycode='{}'".format(country_2,country_1))

df_all = df1.union(df2)

df_all.createOrReplaceTempView("table1")
df_final = sqlSparkContext.sql("SELECT day, COUNT (DISTINCT sourceurl) AS Nb_Events, AVG(averagetone) AS Tone_Mean FROM (table1) GROUP BY day ORDER BY day ASC")
df_final.show()


+--------+---------+-------------------+
|     day|Nb_Events|          Tone_Mean|
+--------+---------+-------------------+
|20110429|        1|-0.9124087691307068|
|20200102|        1| 2.2346367835998535|
|20200107|        1| 3.3057851791381836|
|20200117|        1| 3.4120733737945557|
|20200120|        1| -4.258943557739258|
|20200122|        3|-1.2817181050777435|
|20200213|        1| 1.4758497476577759|
|20200218|        1| 1.0781670808792114|
|20200220|        1|                0.0|
|20200307|        1|-3.1396126747131348|
|20200308|        2|-3.2213603258132935|
|20200311|        1|-4.6728973388671875|
|20200323|        1| -5.761316776275635|
|20200404|        1| -2.826855182647705|
|20200405|        1| -3.995157480239868|
|20200409|        1|-0.6888633966445923|
|20200413|        1|-5.6129984855651855|
|20200419|        1| 1.5193370580673218|
|20200420|        2|  3.332739233970642|
|20200423|        1|-0.4073319733142853|
+--------+---------+-------------------+
only showing top

In [41]:
dfpx = df_final.toPandas()
dfpx['day']= pd.to_datetime(dfpx['day'],format='%Y%m%d')
dfpx.sort_values(by='day',ascending=True,inplace = True)

In [42]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(x=dfpx["day"], y=dfpx["Tone_Mean"], name = 'Ton moyen'),
    secondary_y=False,
)

fig.add_trace(
    go.Bar(x=dfpx["day"], y=dfpx["Nb_Events"], name="Nombre d'articles"),
    secondary_y=True,
)

# Add figure title
fig.update_layout(
    title_text=f"Évolution des relations entre {Country_1} et {Country_2} au cours de l’année"
)

# Set x-axis title
fig.update_xaxes(title_text="Dates")

# Set y-axes titles
fig.update_yaxes(title_text="Ton moyen", secondary_y=False)
fig.update_yaxes(title_text="Nombre d'articles", secondary_y=True)
fig.update_xaxes(rangeslider_visible=True)
fig.show()


### Nouvelles données

In [46]:
date = sqlSparkContext.sql("SELECT max(date) as day FROM mycatalog.gdelt_prod.importstatus").toPandas()
print(pd.to_datetime(date.day,format='%Y%m%d%H%M%S'))

0   2022-02-11 09:45:00
Name: day, dtype: datetime64[ns]


22/02/11 09:47:49 WARN ChannelPool: [s0|/192.168.3.108:9042]  Error while opening new channel (ConnectionInitException: [s0|connecting...] Protocol initialization request, step 1 (STARTUP {CQL_VERSION=3.0.0, DRIVER_NAME=DataStax Java driver for Apache Cassandra(R), DRIVER_VERSION=4.12.0, CLIENT_ID=620b9e75-6972-404e-83f6-3a513d4929c3, APPLICATION_NAME=Spark-Cassandra-Connector-app-20220211083620-0006}): failed to send request (com.datastax.oss.driver.shaded.netty.channel.StacklessClosedChannelException))
22/02/11 09:48:49 WARN ChannelPool: [s0|/192.168.3.108:9042]  Error while opening new channel (ConnectionInitException: [s0|connecting...] Protocol initialization request, step 1 (STARTUP {CQL_VERSION=3.0.0, DRIVER_NAME=DataStax Java driver for Apache Cassandra(R), DRIVER_VERSION=4.12.0, CLIENT_ID=620b9e75-6972-404e-83f6-3a513d4929c3, APPLICATION_NAME=Spark-Cassandra-Connector-app-20220211083620-0006}): failed to send request (com.datastax.oss.driver.shaded.netty.channel.StacklessClose

### Test Shutdown d'un node Cassandra

In [44]:
df1 = sqlSparkContext.sql("SELECT  DISTINCT sourceurl,day, averagetone FROM mycatalog.gdelt_prod.relationship WHERE actor1_geocountrycode='{}' AND actor2_geocountrycode='{}'".format(country_1,country_2))

df2 = sqlSparkContext.sql("SELECT  DISTINCT sourceurl,day, averagetone FROM mycatalog.gdelt_prod.relationship WHERE actor1_geocountrycode='{}' AND actor2_geocountrycode='{}'".format(country_2,country_1))

df_all = df1.union(df2)

df_all.createOrReplaceTempView("table1")
df_final = sqlSparkContext.sql("SELECT day, COUNT (DISTINCT sourceurl) AS Nb_Events, AVG(averagetone) AS Tone_Mean FROM (table1) GROUP BY day ORDER BY day ASC")
df_final.show()

+--------+---------+-------------------+
|     day|Nb_Events|          Tone_Mean|
+--------+---------+-------------------+
|20110429|        1|-0.9124087691307068|
|20200102|        1| 2.2346367835998535|
|20200107|        1| 3.3057851791381836|
|20200117|        1| 3.4120733737945557|
|20200120|        1| -4.258943557739258|
|20200122|        3|-1.2817181050777435|
|20200213|        1| 1.4758497476577759|
|20200218|        1| 1.0781670808792114|
|20200220|        1|                0.0|
|20200307|        1|-3.1396126747131348|
|20200308|        2|-3.2213603258132935|
|20200311|        1|-4.6728973388671875|
|20200323|        1| -5.761316776275635|
|20200404|        1| -2.826855182647705|
|20200405|        1| -3.995157480239868|
|20200409|        1|-0.6888633966445923|
|20200413|        1|-5.6129984855651855|
|20200419|        1| 1.5193370580673218|
|20200420|        2|  3.332739233970642|
|20200423|        1|-0.4073319733142853|
+--------+---------+-------------------+
only showing top

22/02/11 09:35:12 WARN ChannelPool: [s0|/192.168.3.108:9042]  Error while opening new channel (ConnectionInitException: [s0|connecting...] Protocol initialization request, step 1 (STARTUP {CQL_VERSION=3.0.0, DRIVER_NAME=DataStax Java driver for Apache Cassandra(R), DRIVER_VERSION=4.12.0, CLIENT_ID=620b9e75-6972-404e-83f6-3a513d4929c3, APPLICATION_NAME=Spark-Cassandra-Connector-app-20220211083620-0006}): failed to send request (com.datastax.oss.driver.shaded.netty.channel.StacklessClosedChannelException))
22/02/11 09:36:10 WARN ChannelPool: [s0|/192.168.3.108:9042]  Error while opening new channel (ConnectionInitException: [s0|connecting...] Protocol initialization request, step 1 (STARTUP {CQL_VERSION=3.0.0, DRIVER_NAME=DataStax Java driver for Apache Cassandra(R), DRIVER_VERSION=4.12.0, CLIENT_ID=620b9e75-6972-404e-83f6-3a513d4929c3, APPLICATION_NAME=Spark-Cassandra-Connector-app-20220211083620-0006}): failed to send request (com.datastax.oss.driver.shaded.netty.channel.StacklessClose